In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import logging

logging.basicConfig(level=logging.WARNING)

from src.data.VulnerabilityReport import create_from_flama_json # this is the function that creates the VulnerabilityReport object from the json data
from src.ai.LLMService import LLMService # this is the service that uses ollama to generate solution

In [ ]:
# We need the json data, so we are loading it from the file
with open('../data/Dataset.json') as f:
    data = json.load(f)

# Configuration and Data
We 
- get a llm_service object, so it only needs to be created once, making it more efficient. 
- specify the model name, which can be any of the models from here: https://ollama.com/library. 
- specify the number of findings to import. If you want to import all findings, set n=-1.


In [ ]:
model_name=None # but can be anything from here https://ollama.com/library
n=5 # number of findings to import
llm_service = LLMService(model_name=model_name)
vulnerability_report = create_from_flama_json(data, n = n, llm_service=llm_service) # here, we create the VulnerabilityReport object, consisting of a list of Finding objects which each have Solution objects. We pass the llm_service to the VulnerabilityReport object, but this can also just be omitted, in which case it will be created in each Finding object.
vulnerability_report.sort() # this will sort the findings by severity

# AI - 🚀🌕

## First: We add categories to the Findings
This will help us later to improve prompts (hopefully)

In [ ]:
vulnerability_report.add_category() # this will add categories to the findings

## Then we add solutions 🎉
The function below adds long and short solutions to the findings. It also adds keywords for future research.

In [ ]:
vulnerability_report.add_solution(long=True, short=True, search_term=True) # this will add solutions to the findings

# Last but not least: Results
We save and display results in the coming cells

In [ ]:
with open(f'../data/VulnerabilityReport_{n if n !=-1 else "all"}_' + llm_service.get_model_name().replace(":","_") +'.json', 'w') as f:
    f.write(str(json.dumps(vulnerability_report.to_dict())))

In [ ]:
from IPython.display import HTML, Markdown
Markdown(vulnerability_report.to_html(table=True))


In [ ]:
HTML(str(vulnerability_report).replace('\n', '<br />'))
